In [ ]:
%load_ext autoreload
%autoreload 2
import traceback
import pyspark
import datetime
import csv
import time
import gzip
import gc
import sys
import multiprocessing
import pandas as pd
from pprint import pprint
from functools import partial
from pathlib import Path
from typing import Tuple, Optional, List
import lsde2021.utils as utils
import lsde2021.csv as csvutils
import lsde2021.download as dl
import lsde2021.aggregate as agg
from lsde2021.types import PathLike
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
import pyspark.sql.functions as F

In [ ]:
MAX_MEMORY = "60G"

spark = SparkSession \
    .builder \
    .appName("parse-wikipedia-sql-dumps") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

csv_loader = spark.read.format("csv").options(header='True', inferSchema='True')

In [ ]:
def parse_wikipedia_sql_dump(
    sql_input_path: PathLike,
    columns: List[str],
    dest: Optional[PathLike] = None,
    encoding: str = "utf-8",
    nrows: Optional[int] = None,
    log: bool = False,
    write: bool = True,
    force: bool = False
) -> PathLike:
    start = time.time()
    csv_config = dict(
        delimiter=',',
        doublequote=False,
        escapechar='\\',
        quotechar="'",
        strict=True,
    )

    input_path = Path(sql_input_path)
    output_path = input_path.with_suffix(".csv")
    if dest is not None:
        output_path = dest / output_path.name
    
    if not force and output_path.exists():
        print(f"using existing {output_path} ...")
        return output_path

    with utils.fopen(input_path, mode="rt", encoding=encoding, errors="ignore") as input_file:
        output_file = None
        if write:
            output_file = open(output_path, mode="w", encoding=encoding)
        entries = 0
        try:
            writer = None
            if output_file:
                writer = csv.writer(output_file, quoting=csv.QUOTE_MINIMAL)
            if writer:
                # write the header
                writer.writerow(columns)
            
            for line in input_file.readlines():
                # Look for INSERT statement and parse it
                if line.startswith('INSERT INTO'):
                    values = line.partition('` VALUES ')[2]
                    assert values
                    assert values[0] == '('
                    reader = csv.reader([values], **csv_config)
                    for ridx, row in enumerate(reader):
                        for cidx in range(0, len(row), len(columns)):
                            entry = ",".join(row[cidx:cidx+len(columns)])[1:-1]
                            try:
                                entry = tuple(*csv.reader([entry],  delimiter=','))
                                if log:
                                    pprint(entry)
                                    sys.stdout.flush()
                                if writer:
                                    writer.writerow(entry)
                                entries += 1
                            except Exception as e:
                                print("entry:", entry)
                                print("context:", row[cidx:cidx+10])
                                raise e
                            if nrows is not None and entries >= nrows:
                                return output_path
                        gc.collect()
        finally:
            if output_file:
                output_file.close()
                print(f"wrote {entries} rows to {output_path} in {time.time() - start:.2f} seconds ...")
    return output_path

In [ ]:
with utils.fopen("../hdd/wikipedia_sql_dumps/dewiki/20211001/dewiki-20211001-categorylinks.sql.gz") as f:
    print(str(f.read(100)))

In [ ]:
print(utils.detect_encoding(
    "../hdd/wikipedia_sql_dumps/dewiki/20211001/dewiki-20211001-categorylinks.sql.gz"))

In [ ]:
wiki = "enwiki"
for table, cols in [
    ("page", csvutils.PAGE_COLUMNS),
    ("categorylinks", csvutils.CATEGORYLINKS_COLUMNS)
]:
    parse_wikipedia_sql_dump(
        f"../hdd/wikipedia_sql_dumps/{wiki}/20211001/{wiki}-20211001-{table}.sql.gz",
        columns=cols,
        log=True,
        nrows=10,
        force=True,
        write=False,
    )

In [ ]:
dest = Path("../hdd/wikipedia_sql_dumps")
tables = ["langlinks", "page", "category", "categorylinks"]

languages = pd.read_csv("./data/languages.csv", index_col="code")
languages.head()

In [ ]:
downloads = list([
    (table, dest / "/".join(dl.wikimedia_sql_dump_local_file(date, wiki=wiki, table=table)))
    for (date, wiki, table), _ in dl.wikimedia_sql_dump_urls(
        [datetime.date(2021, 10, 1)], wikis=languages["dbname"], tables=tables
    )
])
pprint(downloads[:3])

print("downloaded %d of %d" % (
    len(list(dest.rglob("**/*sql.gz"))), len(downloads)
))

In [ ]:
table_csv_columns = dict(
    page=csvutils.PAGE_COLUMNS,
    category=csvutils.CATEGORY_COLUMNS,
    categorylinks=csvutils.CATEGORYLINKS_COLUMNS,
    langlinks=csvutils.LANGLINKS_COLUMNS,
)

def parse_wikipedia_sql_dump_handler(item: Tuple[str, PathLike], force: bool = False) -> PathLike:
    table, sql_dump_file = item
    csv_columns = table_csv_columns[table]
    return parse_wikipedia_sql_dump(
        sql_dump_file,
        columns=csv_columns,
        force=force,
    )

parsed = sc.parallelize(downloads, numSlices=multiprocessing.cpu_count()).map(
    partial(
        parse_wikipedia_sql_dump_handler,
        force=False,
    )
).collect()

In [ ]:
# check that the parsed files are sane
wiki = "enwiki"
pages = csv_loader.load(
    f"../hdd/wikipedia_sql_dumps/{wiki}/20211001/{wiki}-20211001-page.sql.csv"
).limit(10_000)
pages.toPandas().head()

In [ ]:
categories = csv_loader.load(
    f"../hdd/wikipedia_sql_dumps/{wiki}/20211001/{wiki}-20211001-category.sql.csv"
).limit(10_000)
print(categories.count(), "rows")
categories.toPandas().head()

In [ ]:
categorylinks = csv_loader.load(
    f"../hdd/wikipedia_sql_dumps/{wiki}/20211001/{wiki}-20211001-categorylinks.sql.csv"
).limit(10_000)
categorylinks.toPandas().head()

In [ ]:
langlinks = csv_loader.load(
    f"../hdd/wikipedia_sql_dumps/{wiki}/20211001/{wiki}-20211001-langlinks.sql.csv"
).limit(10_000)
langlinks.toPandas().head()

In [ ]:
#### Parse CSV files and convert to correct data dypes using chunking
wikis = languages["dbname"]
wikis = [w for w in wikis if w != "enwiki"]
assert "enwiki" not in wikis
parsed = list([
    (table, (dest / "/".join(dl.wikimedia_sql_dump_local_file(date, wiki=wiki, table=table))).with_suffix(".csv"))
    for (date, wiki, table), _ in dl.wikimedia_sql_dump_urls(
        [datetime.date(2021, 10, 1)], wikis=wikis, tables=tables
    )
])
pprint(parsed[:5])

In [ ]:
def try_to_int(val):
    try:
        return int(val)
    except ValueError:
        return np.nan
    
def try_to_float(val):
    try:
        return float(val)
    except ValueError:
        return np.nan

In [ ]:
def prepare_page_sql_dump(parsed_dump_file: PathLike, output_path: PathLike, chunksize: int = 10 ** 4, max_chunks: Optional[int] = None) -> pd.DataFrame:
    header = True
    with csvutils.read_page_csv(
        parsed_dump_file,
        engine="c",
        dtype=csvutils.RAW_PAGE_DTYPE,
        chunksize=chunksize,
    ) as reader:
        for n, chunk in enumerate(reader):
            # print("before")
            # print(chunk.head())
            chunk["page_id"] = pd.to_numeric(chunk["page_id"], errors='coerce', downcast="unsigned")
            chunk["page_namespace"] = pd.to_numeric(chunk["page_namespace"], errors='coerce', downcast="unsigned")
            chunk["page_title"] = chunk["page_title"].astype("string")
            chunk["page_restrictions"] = chunk["page_restrictions"].astype("string")

            chunk["page_is_redirect"] = chunk["page_is_redirect"].astype("bool")
            chunk["page_is_new"] = chunk["page_is_new"].astype("bool")

            chunk["page_random"] = pd.to_numeric(chunk["page_random"], errors='coerce', downcast="unsigned")
            chunk["page_touched"] = pd.to_datetime(chunk["page_touched"], errors='coerce')
            chunk["page_links_updated"] = pd.to_datetime(chunk["page_links_updated"], errors='coerce')

            chunk["page_len"] = pd.to_numeric(chunk["page_len"], errors='coerce', downcast="unsigned")
            chunk["page_content_model"] = chunk["page_content_model"].astype("category")
            chunk["page_lang"] = chunk["page_lang"].astype("category")
            # print("after")
            # print(chunk.head())
            
            chunk.to_csv(output_path, header=header, mode='a')
            header = False
            if max_chunks is not None and n >= max_chunks:
                break
    return output_path

In [ ]:
def prepare_categorylinks_sql_dump(parsed_dump_file: PathLike, output_path: PathLike, chunksize: int = 10 ** 2, max_chunks: Optional[int] = None) -> pd.DataFrame:
    header = True
    with csvutils.read_categorylinks_csv(
        parsed_dump_file,
        engine="c",
        dtype=csvutils.RAW_CATEGORYLINKS_DTYPE,
        chunksize=chunksize,
    ) as reader:
        for n, chunk in enumerate(reader):
            # print("before")
            # print(chunk.head())
            chunk["page_id"] = pd.to_numeric(chunk["page_id"].astype("string"), errors='coerce', downcast="unsigned")
            chunk["category_name"] = chunk["category_name"].astype("string")
            chunk["sortkey"] = chunk["sortkey"].astype("category")
            chunk["timestamp"] = pd.to_datetime(chunk["timestamp"], errors='coerce')
            chunk["sortkey_prefix"] = chunk["sortkey_prefix"].astype("category")
            chunk["collation"] = chunk["collation"].astype("category")
            chunk["type"] = chunk["type"].astype("category")
            # print("after")
            # print(chunk.head())
            chunk.to_csv(output_path, header=header, mode='a')
            header = False
            if max_chunks is not None and n >= max_chunks:
                break
    return output_path

In [ ]:
def prepare_category_sql_dump(parsed_dump_file: PathLike, output_path: PathLike, chunksize: int = 10 ** 4, max_chunks: Optional[int] = None) -> pd.DataFrame:
    header = True
    with csvutils.read_category_csv(
        parsed_dump_file,
        engine="c",
        dtype=csvutils.RAW_CATEGORY_DTYPE,
        chunksize=chunksize,
    ) as reader:
        for n, chunk in enumerate(reader):
            # print("before")
            # print(chunk.head())
            chunk["cat_id"] = pd.to_numeric(chunk["cat_id"], errors='coerce', downcast="unsigned")
            chunk["cat_title"] = chunk["cat_title"].astype("string")
            chunk["cat_pages"] = pd.to_numeric(chunk["cat_pages"], errors='coerce', downcast="unsigned")
            chunk["cat_subcats"] = pd.to_numeric(chunk["cat_subcats"], errors='coerce', downcast="unsigned")
            chunk["cat_files"] = pd.to_numeric(chunk["cat_files"], errors='coerce', downcast="unsigned")
            # print("after")
            # print(chunk.head())
            
            chunk.to_csv(output_path, header=header, mode='a')
            header = False
            if max_chunks is not None and n >= max_chunks:
                break
    return output_path

In [ ]:
def prepare_langlinks_sql_dump(parsed_dump_file: PathLike, output_path: PathLike, chunksize: int = 10 ** 4, max_chunks: Optional[int] = None) -> pd.DataFrame:
    header = True
    with csvutils.read_langlinks_csv(
        parsed_dump_file,
        engine="c",
        dtype=csvutils.RAW_LANGLINKS_DTYPE,
        chunksize=chunksize,
    ) as reader:
        for n, chunk in enumerate(reader):
            # print("before")
            # print(chunk.head())
            chunk["page_id"] = pd.to_numeric(chunk["page_id"], errors='coerce', downcast="unsigned")
            chunk["lang"] = chunk["lang"].astype("category")
            chunk["lang_title"] = chunk["lang_title"].astype("string")
            # print("after")
            # print(chunk.head())
            
            chunk.to_csv(output_path, header=header, mode='a')
            header = False
            if max_chunks is not None and n >= max_chunks:
                break
    return output_path

In [ ]:
def prepare_wikipedia_sql_dump_handler(item: Tuple[str, PathLike], force: bool = False, max_chunks: Optional[int] = None) -> PathLike:
    table, parsed_sql_dump_file = item
    output_path = utils.strip_extension(parsed_sql_dump_file)
    output_path = (output_path.parent / (output_path.stem + "-converted")).with_suffix(".sql.csv")
    print(table, parsed_sql_dump_file, output_path)
    
    if not force and output_path.exists():
        print(f"using existing {output_path} ...")
        return output_path
    
    # since we parse in chunks and append to the file, we have to remove it first
    output_path.unlink(missing_ok=True)
    print(f"creating {output_path} ...")
    
    start = time.time()
    if table == "page":
        prepare_page_sql_dump(parsed_sql_dump_file, output_path, max_chunks=max_chunks)
    if table == "categorylinks":
        prepare_categorylinks_sql_dump(parsed_sql_dump_file, output_path, max_chunks=max_chunks)
    if table == "category":
        prepare_category_sql_dump(parsed_sql_dump_file, output_path, max_chunks=max_chunks)
    if table == "langlinks":
        prepare_langlinks_sql_dump(parsed_sql_dump_file, output_path, max_chunks=max_chunks)
    
    print(f"created {output_path} in {time.time() - start:.2f} seconds ...")
    return output_path

parallel = multiprocessing.cpu_count()
converted = sc.parallelize(parsed, numSlices=parallel).map(
    partial(
        prepare_wikipedia_sql_dump_handler,
        force=False,
        # max_chunks=10,
    )
).collect()

In [ ]:
# check out the namespaces in more detail
all_namespaces = pages.select("page_namespace").distinct()
all_namespaces.show()
print(all_namespaces.count(), "page_namespace")

In [ ]:
csv_loader.load()

In [ ]:
# find the german wikipedia artices with no translation entry
german_pages = csv_loader.load(
    f"../hdd/wikipedia_sql_dumps/dewiki/20211001/dewiki-20211001-page.sql.csv"
).limit(10_000)

page_langlinks = pages.join(
    langlinks.select([
        F.col("lang").alias("id"),
        F.col("lang_title").alias("translation_lang"),
        F.col("page_id").alias("english_page_id")
    ]), on="id", how="outer")
page_langlinks.show()


without_translation = page_langlinks.filter(F.col("translation_lang").isNull())

print("total:", page_langlinks.count())
print("without translation:", without_translation.count())